In [ ]:
import numpy as np
import pandas as pd
import csv
import pandasql as ps
import sqlite3
import os

import datetime
from datetime import datetime

In [ ]:
#report = pd.read_csv('/home/xuranzeng/project/data/crimes_data_zone_simplified.csv')
#report = report.loc[(report["day"]>"2018-08-17") & (report["day"]<="2019-08-17")]

In [ ]:
# load bpc and report
report = pd.read_csv('/home/xuranzeng/project/data/crimes_data_zone_simplified_sample.csv')
report.drop(report.iloc[:, 0:2], inplace = True, axis = 1)

bpc_freq = pd.read_csv('/project/graziul/features/meta/bpc_freq.csv')
bpc_freq = bpc_freq.loc[(bpc_freq["date"]>"2018_08_17") & (bpc_freq["date"]<="2019_08_17")]

In [ ]:
# process time
bpc_freq['start'] = pd.to_datetime(bpc_freq['file'], format='%Y%m%d%H%M')

# process zone
bpc_freq['zone'] = pd.to_numeric(bpc_freq['zone'].str[4:])

# sort
bpc_freq = bpc_freq.sort_values(by=['start',"zone"])

# generate 30 min time range
bpc_freq['end'] = bpc_freq.groupby('zone')['start'].shift(-1)

# process time
report['Date2'] = pd.to_datetime(report['Date'], format='%m/%d/%Y %I:%M:%S %p')

# process zone
report['Zone'] = pd.to_numeric(report['Zone'])

# sort
report = report.sort_values(by=['Date2',"Zone"])

In [ ]:
# zone 1
bpc_freq_sub = bpc_freq.loc[bpc_freq["zone"]==1]
report_sub = report.loc[report["Zone"]==1]

In [ ]:
# time align bpc file and report
#Make the db in memory
conn = sqlite3.connect(':memory:')

#write the tables
bpc_freq_sub.to_sql('bpc_freq_sub', conn, index=False)
report_sub.to_sql('report_sub', conn, index=False)

qry = '''
    select  
        *
    from
        bpc_freq_sub join report_sub 
        on  Date2 between start and end and
        bpc_freq_sub.zone = report_sub.Zone
    '''

df = pd.read_sql_query(qry, conn)

In [ ]:
#df.to_csv('/home/xuranzeng/project/data/aligned_sample.csv')

In [ ]:
# method 2 (slow)
#df = pd.merge(bpc_merge,report_merge, how='left', left_on='zone', right_on='Zone')
#df = df[(df['Date2']>=df['start']) & (df['Date2']<=df['end'])]